In [1]:
%pip install -qU langgraph langchain langchain_openai langchain_experimental pymupdf tiktoken python-dotenv

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
# from helper.file_loader import FileLoader

load_dotenv()

open_ai_key = os.getenv("OPENAI_API_KEY")
langchain_key = os.getenv("LANGCHAIN_API_KEY")

llm = ChatOpenAI(api_key=open_ai_key, model="gpt-4o")

In [3]:
from langchain.document_loaders import PyMuPDFLoader

document = PyMuPDFLoader("assets/real_estate_book_practice.pdf").load()

In [4]:
import tiktoken
from langchain.text_splitter import RecursiveCharacterTextSplitter

def tiktoken_len(text):
    tokens = tiktoken.encoding_for_model("gpt-4o-mini").encode(
        text,
    )
    return len(tokens)

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 300,
    chunk_overlap = 0,
    length_function = tiktoken_len,
)

split_chunks = text_splitter.split_documents(document)

In [5]:
from langchain_openai.embeddings import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

In [6]:
%pip install -qU --disable-pip-version-check qdrant-client

Note: you may need to restart the kernel to use updated packages.


In [7]:
from langchain_community.vectorstores import Qdrant

qdrant_vectorstore = Qdrant.from_documents(
  split_chunks,
  embedding_model,
  location=":memory:",
  collection_name="extending_context_window_llama_3",
)

In [8]:
qdrant_retriever = qdrant_vectorstore.as_retriever()

In [9]:
from langchain_core.prompts import ChatPromptTemplate

RAG_PROMPT = """
CONTEXT: 
{context}

QUERY: 
{question}

You are a Real Estate Agent with 20 years of experience that enjoys mentoring your clients. Use the available context to answer the question. If you can't answer the question say you don't know.
"""

rag_prompt = ChatPromptTemplate.from_template(RAG_PROMPT)

In [10]:
from operator import itemgetter
from langchain.schema.output_parser import StrOutputParser

rag_chain = (
  {"context": itemgetter("question") | qdrant_retriever, "question": itemgetter("question")}
  | rag_prompt | llm | StrOutputParser()
)

In [13]:
#testing the chain
rag_chain.invoke({"question": "what are the fiduciary duties of a real estate agent?"})

"In a real estate transaction, the fiduciary duties of an agent are critical responsibilities that the agent owes to their client. These duties require the agent to act in the highest good faith towards their client. Specifically, the fiduciary duties include:\n\n1. **Loyalty and Good Faith**: The agent must act in the best interests of the client and not obtain any advantage over the client through misrepresentation, concealment, duress, or undue influence.\n\n2. **Utmost Care, Integrity, and Honesty**: The agent is expected to maintain high standards of care, integrity, and honesty while dealing with the client and all parties involved in the transaction.\n\n3. **Disclosure**: The agent must disclose all relevant information that affects the value and desirability of the property, ensuring fair dealing and good faith in their interactions.\n\nThese duties are essential to maintain trust and ensure that the client's interests are protected throughout the real estate transaction."